In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE146361"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE146361"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE146361.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE146361.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE146361.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyze the output to determine the dataset characteristics

# 1. Gene Expression Data Availability
# From the background information, we can see this dataset contains gene expression data
# It mentions "gene expression profile" and "HumanHT-12 v3.0 Expression BeadChip array (Illumina)"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For the trait (Stomach Cancer), we can see all samples have "disease: Gastric Cancer" (key 0)
trait_row = 0

# For age, there's no information available
age_row = None

# For gender, there's no information available
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (1 for disease, 0 for control)"""
    if not isinstance(value, str):
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # All samples have gastric cancer, so all will be 1
    if "gastric cancer" in value.lower():
        return 1
    return None

def convert_age(value):
    """Convert age value to continuous"""
    # Not used as age data is unavailable
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used as gender data is unavailable
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # Based on the sample characteristics, we know the dataset contains 27 cell lines
    # Each cell line has the same disease status (Gastric Cancer)
    
    # Create the clinical data DataFrame
    sample_chars = {
        0: ['disease: Gastric Cancer'], 
        1: ['organism part: Stomach'], 
        2: ['cell line: Gastric Cancer Cell line'], 
        3: ['cell line: Hs746T', 'cell line: YCC-16', 'cell line: YCC-2', 'cell line: SNU-16', 
            'cell line: SNU-719', 'cell line: YCC-9', 'cell line: SNU-668', 'cell line: MKN-74', 
            'cell line: SNU-1', 'cell line: SNU-5', 'cell line: MKN-45', 'cell line: SNU-638', 
            'cell line: SNU-216', 'cell line: YCC-6', 'cell line: YCC-7', 'cell line: MKN-1', 
            'cell line: MKN-28', 'cell line: NCI-N87', 'cell line: SNU-484', 'cell line: SNU-601', 
            'cell line: SNU-620', 'cell line: YCC-3', 'cell line: YCC-11', 'cell line: YCC-1', 
            'cell line: AGS', 'cell line: KATOIII', 'cell line: SNU-520']
    }
    
    # Extract cell line names from the sample characteristics
    cell_lines = [line.split(": ")[1] for line in sample_chars[3]]
    
    # Create a DataFrame with all samples having the same trait value
    clinical_data = pd.DataFrame(index=cell_lines)
    
    # Add sample characteristics as columns
    for row_idx, values in sample_chars.items():
        # Handle the case where row 3 has multiple values (one per cell line)
        if row_idx == 3:
            continue  # Skip as we've already used these to create the index
        
        # For other rows, all cell lines share the same value
        for value in values:
            # Use the part before colon as column name, and after colon as value
            if ":" in value:
                col_name, val = value.split(":", 1)
                clinical_data[col_name.strip()] = val.strip()
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    print("Preview of selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save the data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Based on biomedical knowledge, these are Illumina probe IDs (indicated by the "ILMN_" prefix)
# and not human gene symbols. These probe IDs need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine the appropriate columns for mapping
# Based on the preview of gene_annotation, we can see:
# - 'ID' contains the Illumina probe IDs (same as in gene expression data)
# - 'Symbol' contains the gene symbols we need

# 2. Get the gene mapping dataframe using the specified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print a preview of the mapping
print("Preview of gene mapping:")
print(gene_mapping.head())
print(f"Gene mapping shape: {gene_mapping.shape}")

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the resulting gene expression data
print("\nPreview of mapped gene expression data:")
print(gene_data.head())
print(f"Gene expression data shape after mapping: {gene_data.shape}")

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nPreview after normalizing gene symbols:")
print(gene_data.head())
print(f"Gene expression data shape after normalization: {gene_data.shape}")

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Load the normalized gene data that was saved in the previous step
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded normalized gene data shape: {gene_data.shape}")
except Exception as e:
    print(f"Error loading gene data: {e}")
    raise ValueError("Cannot proceed without gene expression data")

# 2. Load the clinical data created in Step 2
try:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data shape: {clinical_df.shape}")
    print("Clinical data preview:")
    print(clinical_df.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()

# The sample IDs in the gene expression data are GSM IDs
gene_sample_ids = gene_data.columns
print(f"Gene data has {len(gene_sample_ids)} samples")
print(f"First few sample IDs: {list(gene_sample_ids[:5])}")

# Create a clinical DataFrame with the same sample IDs as the gene data
# We know from previous steps that all samples are cancer cell lines (all have trait=1)
clinical_features = pd.DataFrame(index=gene_sample_ids)
clinical_features[trait] = 1  # All samples are gastric cancer cell lines

print(f"Prepared clinical features shape: {clinical_features.shape}")
print(clinical_features.head())

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 5. Conduct quality check and save the cohort information
note = "Dataset contains gene expression data from 27 gastric cancer cell lines. All samples have the same trait value (all are cancer), making the dataset unsuitable for associative studies."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 6. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. The gene data was already normalized in Step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded normalized gene data shape: {gene_data.shape}")

# 2. Create clinical features for the same samples that are in the gene data
# From previous steps, we know all samples are gastric cancer cell lines
clinical_features = pd.DataFrame(index=gene_data.columns)
clinical_features[trait] = 1  # All samples are gastric cancer
print(f"Prepared clinical features shape: {clinical_features.shape}")
print(clinical_features.head())

# Direct approach to link clinical and genetic data
linked_data = clinical_features.copy()
# Add gene expression data as additional columns
for gene in gene_data.index:
    linked_data[gene] = gene_data.loc[gene]
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data_processed = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_processed.shape}")

# 4. Determine whether the trait is biased
is_trait_biased, linked_data_processed = judge_and_remove_biased_features(linked_data_processed, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data_processed.shape}")

# 5. Conduct quality check and save cohort information
note = "Dataset contains gene expression data from 27 gastric cancer cell lines. All samples have the same trait value (all are cancer), making the dataset unsuitable for case-control associative studies."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data_processed,
    note=note
)

# 6. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_processed.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")